In [5]:
import numpy as np
from scipy import stats
from operator import itemgetter

##import values from Excel spreadsheet/google spreadsheet
##a[a[:,1].argsort()] to sort by second column
##ind = np.lexsort((a[:,1],a[:,0])) to sort by two fields
##np.arange(12,24).reshape(3,4) - may be able to use to reshape a list into a matrix

class pool: 
    def __init__(self, court_num, team_names):
        self.court = court_num ## name of court e.g. M1, W1, etc
        self.teams = team_names ## list/tuple of team names
        self.points = np.zeros(len(self.teams), dtype=np.int)
        self.HTH = {}
        self.games_won = np.zeros(len(self.teams), dtype=np.int)
        self.notes = ''
        self.rankings = []
        self.rankings_sorted = []
        
    def get_teams(self):
        return self.teams
    
    def get_teamindex(self,team_name):
        return self.teams.index(team_name)
    
    def get_points(self, team_name=0):
        if team_name==0:
            return self.points
        else:
            if team_name in self.teams:
                index = self.get_teamindex(team_name)
                return self.points[index]
            else:
                return "Team not found"
    
    def get_gameswon(self, team_name=0):
        if team_name==0:
            return self.games_won
        else:
            if team_name in self.teams:
                index = self.get_teamindex(team_name)
                return self.games_won[index]
            else:
                return "Team not found"
        
    def add_notes(self, comment): ##for storing comments about lost ball, etc
        self.notes += comment
    
    def get_notes(self):
        return self.notes
        
    def add_games(self, team_names, scores): ##team_names - list of team names, scores - list of scores as tuples, in order of team names
        game_results = {team_names[0]:0, team_names[1]:0}
        pt_results = {team_names[0]:0, team_names[1]:0}
        
        for score in scores:
            diff = score[1] - score[0]
            if diff > 0:
                game_results[team_names[1]] += 1
            elif diff < 0:
                game_results[team_names[0]] += 1
            else:
                print ("Scores are the same:" + score[0] + ":" + score[1])
            
            pt_results[team_names[1]] += diff
            pt_results[team_names[0]] -= diff
        
        index0 = self.teams.index(team_names[0])
        index1 = self.teams.index(team_names[1])
        
        self.points[index0] += pt_results[team_names[0]]
        self.points[index1] += pt_results[team_names[1]]
        
        self.games_won[index0] += game_results[team_names[0]]
        self.games_won[index1] += game_results[team_names[1]]
        
        HTH_key=[index0,index1]
        HTH_key.sort()
        
        self.HTH[tuple(HTH_key)] = pt_results[self.teams[HTH_key[1]]]
        
    def get_HTH(self, indexes=0): ##positive number means team with higher index WON point diff
        if indexes == 0:
            return self.HTH
        else:
            indexes.sort()
            return self.HTH[tuple(indexes)]
    
    def resolve_HTH(self):
        ##for resolving HTH when the games won are the same
        ##will only change order if it's wrong
        ##anytime rankings are resorted, HTH needs to be resolved again
        for i in range(4):
            if self.rankings_sorted[i][2] == self.rankings_sorted[i+1][2]:
                indexes = [self.rankings_sorted[i][0],self.rankings_sorted[i+1][0]]
                if tuple(indexes) in self.HTH.keys():
                    if self.get_HTH(indexes) > 0:
                        ##need to switch the order
                        copy = self.rankings_sorted[i][:]
                        self.rankings_sorted[i] = self.rankings_sorted[i+1]
                        self.rankings_sorted[i+1] = copy
                    else:
                        if self.get_HTH(indexes) == 0:
                            print ("HTH is tied")
                else:
                    indexes.sort()
                    if self.get_HTH(indexes) < 0:
                        copy = self.rankings_sorted[i][:]
                        self.rankings_sorted[i] = self.rankings_sorted[i+1]
                        self.rankings_sorted[i+1] = copy
                    else:
                        if self.get_HTH(indexes) == 0:
                            print ("HTH is tied")
    def resolve_ties(self):
        modes = stats.mode(self.rankings)
        if modes[1][0][2] == 2:
            self.resolve_HTH()
        #how to resolve 3-way ties
            
    def get_unsorted_rankings(self): ##outputs list of teams **not ranked
        ##use tuples, (index, team_name, games_won, points)
        team_stats = []
        for i in range(len(self.teams)):
            team_stats.append((i, self.teams[i], self.games_won[i], self.points[i]))
            
        self.rankings = team_stats
        return self.rankings
    
    def get_sorted_rankings(self):
        self.rankings_sorted = self.get_unsorted_rankings()
        self.rankings_sorted = sorted(self.rankings_sorted, key=itemgetter(2,3), reverse=True)
        self.resolve_ties() #which should include resolve_HTH if applicable
        return self.rankings_sorted
    
    def pp_rankings(self, sort=0): ##pretty print rankings
        if sort == 0:
            rankings = self.rankings
        else:
            rankings = self.rankings_sorted
        
        print('\nPool:' + self.court)
        print ('Team Name\tGames Won\tPoints Overall')
        
        for item in rankings:
            print(item[1], '\t\t', item[2], '\t\t', item[3]) 
            
class tourney():
    def __init__(self, pools, brackets, pool_size=4):
        self.pools = pools #input: list of pool objects that below to this tourney
        self.brackets = brackets #list with three ints - e.g [16,16,32] for number of brackets and how many teams in each
        self.pool_size = pool_size
        self.rankings = []
        self.gold = []
        self.silver = []
        self.bronze = []
        
    def get_overall_rankings(self): #grab ranking info from pools
        for pool in self.pools:
            self.rankings.append(pool.get_sorted_rankings)
            
        self.rankings = sorted(self.rankings_sorted, key=itemgetter(2,3), reverse=True)
        
        
    def fill_bracket(self): #determine all bracket results
        #fill up the arrays in order, then transpose every other row
        for i in np.arange(1,len(self.rankings),self.brackets[0]/self.pool_size):
            
        
    def pp_bracket(self, bracket_num=0): ##pretty print the bracket info. bracket num: 0=all, 1= gold, etc
        ##need another way to print bracket visually. also need to track bracket info?
        print(self.rankings)
        
        
def main():
    ##Step1: create team names to feed into pools
    ##Step2: Create pools from list of team names
    ##Step3: Add games
    ##Step4: Print rankings by pool
    
    ##Step1: Create tournament
    ##Step2: Decide Gold, Silver, Bronze brackets
    
    teams = ['teama', 'teamb','teamc', 'teamd', 'teame']
    poolM1 = pool('M1', teams)
    poolM1.add_games(['teama','teamb'], [[21,15],[19,21]])
    poolM1.add_games(['teama','teamc'], [[22,20],[21,19]])
    poolM1.add_games(['teama','teamd'], [[22,20],[21,19]])
    poolM1.add_games(['teama','teame'], [[22,20],[21,19]])
    
    poolM1.add_games(['teamb','teamc'], [[22,10],[21,19]])
    poolM1.add_games(['teamb','teamd'], [[22,10],[21,19]])
    poolM1.add_games(['teamb','teame'], [[22,10],[21,19]])
    
    poolM1.add_games(['teamc','teamd'], [[22,24],[23,21]])
    poolM1.add_games(['teamc','teame'], [[22,24],[23,21]])
    
    poolM1.add_games(['teamd','teame'], [[22,20],[23,21]])
    
    poolM1.get_unsorted_rankings()
    poolM1.get_sorted_rankings()
    poolM1.pp_rankings(1)
    
    TNMT = tourney([poolM1],[16,16,32])
    TNMT.fill_bracket()
    TNMT.pp_bracket()
        
main()


Pool:M1
Team Name	Games Won	Points Overall
teama 		 7 		 16
teamb 		 7 		 38
teamd 		 3 		 -14
teamc 		 2 		 -18
teame 		 1 		 -22
[]


C:\Users\lye\AppData\Local\Continuum\Anaconda3\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
